In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile ,os

In [3]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import random
import numpy as np
import shutil
import os
import cv2
import imgaug as ia
from imgaug import augmenters as iaa
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

In [ ]:
! mkdir '/content/drive/MyDrive/seokwon/train_real'

In [ ]:
import os
import zipfile

zip_file_path = '/content/drive/MyDrive/seokwon/train_seg.zip'
extract_path = '/content/drive/MyDrive/seokwon/train_real'

with zipfile.ZipFile(zip_file_path, 'r') as zf:
    zipinfo = zf.infolist()
    for member in zipinfo:
        if member.filename.endswith('.txt'):
            zf.extract(member, path=extract_path)
            file_path = os.path.join(extract_path, member.filename)
            os.remove(file_path)
        else:
            zf.extract(member, path=extract_path)

In [4]:
data='/content/drive/MyDrive/seokwon/train_real'
categories = os.listdir(data)
for category in categories:
    folder = os.path.join(data,category)
    name=os.listdir(folder)
    print(f'{category}:{len(name)}')

가구수정:500
걸레받이수정:500
곰팡이:500
꼬임:500


KeyboardInterrupt: ignored

In [ ]:
# import cv2
# import numpy as np
# import os

# data_path = "/content/train"

# categories = os.listdir(data_path)
# labels = [i for i in range(len(categories))]

# label_dict = dict(zip(categories, labels)) 

# img_size = 224

# data = []
# target = []

# for category in categories:
#     folder_path = os.path.join(data_path, category)
#     img_names = os.listdir(folder_path)

#     for img_name in img_names:
#         img_path = os.path.join(folder_path, img_name)
#         img = cv2.imread(img_path)

#         try:
#             resized = cv2.resize(img, (img_size, img_size))
#             data.append(resized)
#             target.append(label_dict[category])

#         except Exception as e:
#             print('Exception:', e)

In [4]:
data_path = "/content/drive/MyDrive/seokwon/train_real"
# output_path = "/content/drive/MyDrive/seokwon/train_real"

In [ ]:
categories = os.listdir(data_path)
labels = [i for i in range(len(categories))]
label_dict = dict(zip(categories, labels))

In [ ]:
# ! mkdir '/content/drive/MyDrive/seokwon/undersampled'
# ! mkdir '/content/drive/MyDrive/seokwon/argumented'

In [ ]:
img_size = 224

seq = iaa.Sequential([
    iaa.Fliplr(0.5),
    iaa.Crop(percent=(0, 0.1)),
    iaa.ContrastNormalization((0.9, 2.0)),
    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),
    iaa.Multiply((0.8, 1.2), per_channel=0.2),
    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True)

/usr/local/lib/python3.10/dist-packages/imgaug/imgaug.py:184: DeprecationWarning: Function `ContrastNormalization()` is deprecated. Use `imgaug.contrast.LinearContrast` instead.
  warn_deprecated(msg, stacklevel=3)


In [ ]:
for category in categories:
    folder = os.path.join(data_path, category)
    img_names = sorted(os.listdir(folder))
    img_count = len(img_names)

    if img_count >= 500:
        print(f"Skip augmenting {category} - already has {img_count} images")
        continue  # 이미지 수가 1405개 이상이면 증강하지 않고 다음 카테고리로 이동

    aug_per_image = int(np.ceil((500 - img_count) / img_count))

    class_count = img_count
    for i, img_name in enumerate(img_names):
        img_path = os.path.join(folder, img_name)
        img = cv2.imread(img_path)
        resized = cv2.resize(img, (img_size, img_size))

        for j in range(1, aug_per_image + 1):
            output_filename = f"{img_name.split('.')[0]}_aug{j}.png"
            output_file_path = os.path.join(folder, output_filename)
            aug_img = seq.augment_image(resized)
            cv2.imwrite(output_file_path, aug_img)
            class_count += 1
            if class_count >= 500:
                break

        if class_count >= 500:
            break

Skip augmenting 오염 - already has 595 images
Skip augmenting 훼손 - already has 1405 images


In [5]:
data='/content/drive/MyDrive/seokwon/train_real'
categories = os.listdir(data)
for category in categories:
    folder = os.path.join(data,category)
    name=os.listdir(folder)
    print(f'{category}:{len(name)}')

가구수정:500
걸레받이수정:500
곰팡이:500
꼬임:500
녹오염:500
들뜸:500
면불량:500
몰딩수정:500
반점:500
석고수정:500
오염:595
오타공:500
울음:500
이음부불량:500
창틀,문틀수정:500
터짐:500
틈새과다:500
피스:500
훼손:1405


In [6]:
! mkdir '/content/drive/MyDrive/seokwon/under'

In [7]:
data_path = '/content/drive/MyDrive/seokwon/train_real' 
undersampled_path = '/content/drive/MyDrive/seokwon/under' 

In [8]:
for category in categories:
    folder = os.path.join(data_path, category)
    img_names = sorted(os.listdir(folder))
    img_count = len(img_names)

    if img_count > 500:
        # 언더샘플링
        undersampled_files = random.sample(img_names, 500)
        for file in undersampled_files:
            file_path = os.path.join(folder, file)
            save_path = os.path.join(undersampled_path, category, file)
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            shutil.copy(file_path, save_path)
    else:
        # 원본 파일 복사
        for file in img_names:
            file_path = os.path.join(folder, file)
            save_path = os.path.join(undersampled_path, category, file)
            os.makedirs(os.path.dirname(save_path), exist_ok=True)
            shutil.copy(file_path, save_path)

In [25]:
data='/content/drive/MyDrive/seokwon/under'
categories = os.listdir(data)
for category in categories:
    folder = os.path.join(data,category)
    name=os.listdir(folder)
    print(f'{category}:{len(name)}')

가구수정:500
걸레받이수정:500
곰팡이:500
꼬임:500
녹오염:500
들뜸:500
면불량:500
몰딩수정:500
반점:500
석고수정:500
오염:500
오타공:500
울음:500
이음부불량:500
창틀,문틀수정:500
터짐:500
틈새과다:500
피스:500
훼손:500


In [12]:
! mkdir '/content/drive/MyDrive/seokwon/train_tm/' ! mkdir '/content/drive/MyDrive/seokwon/train_tm/틈'

mkdir: cannot create directory ‘!’: File exists
mkdir: cannot create directory ‘mkdir’: File exists


In [13]:
! mkdir '/content/drive/MyDrive/seokwon/train_tm/얼룩' ! mkdir '/content/drive/MyDrive/seokwon/train_tm/주름'

mkdir: cannot create directory ‘!’: File exists
mkdir: cannot create directory ‘mkdir’: File exists


In [14]:
source_folder = '/content/drive/MyDrive/seokwon/under'
target_folder = '/content/drive/MyDrive/seokwon/train_tm/틈'

categories = ['가구수정', '걸레받이수정', '들뜸', '몰딩수정', '창틀,문틀수정', '틈새과다']

for category in categories:
    category_folder = os.path.join(source_folder, category)
    files = os.listdir(category_folder)

    for file in files:
        source_path = os.path.join(category_folder, file)
        target_path = os.path.join(target_folder, category, file)
        os.makedirs(os.path.dirname(target_path), exist_ok=True)
        shutil.copy(source_path, target_path)

In [26]:
target_folder = '/content/drive/MyDrive/seokwon/train_tm/틈'

file_count = sum([len(files) for _, _, files in os.walk(target_folder)])

print("틈 폴더 안에 있는 파일 개수:", file_count)

틈 폴더 안에 있는 파일 개수: 3000


In [16]:
source_folder = '/content/drive/MyDrive/seokwon/under'
target_folder = '/content/drive/MyDrive/seokwon/train_tm/얼룩'

categories = ['곰팡이', '녹오염', '반점', '오염']

for category in categories:
    category_folder = os.path.join(source_folder, category)
    files = os.listdir(category_folder)

    for file in files:
        source_path = os.path.join(category_folder, file)
        target_path = os.path.join(target_folder, category, file)
        os.makedirs(os.path.dirname(target_path), exist_ok=True)
        shutil.copy(source_path, target_path)

In [27]:
target_folder = '/content/drive/MyDrive/seokwon/train_tm/얼룩'

file_count = sum([len(files) for _, _, files in os.walk(target_folder)])

print("얼룩 폴더 안에 있는 파일 개수:", file_count)

얼룩 폴더 안에 있는 파일 개수: 2000


In [18]:
source_folder = '/content/drive/MyDrive/seokwon/under'
target_folder = '/content/drive/MyDrive/seokwon/train_tm/주름'

categories = ['꼬임','면불량','석고수정','오타공','울음','이음부불량','터짐','피스','훼손']

for category in categories:
    category_folder = os.path.join(source_folder, category)
    files = os.listdir(category_folder)

    for file in files:
        source_path = os.path.join(category_folder, file)
        target_path = os.path.join(target_folder, category, file)
        os.makedirs(os.path.dirname(target_path), exist_ok=True)
        shutil.copy(source_path, target_path)

In [28]:
target_folder = '/content/drive/MyDrive/seokwon/train_tm/주름'

file_count = sum([len(files) for _, _, files in os.walk(target_folder)])

print("주름 폴더 안에 있는 파일 개수:", file_count)

주름 폴더 안에 있는 파일 개수: 4500


In [ ]:
# data_path = '/content/drive/MyDrive/seokwon/train_1'
# train_path = '/content/drive/MyDrive/seokwon/train_3'
# valid_path = '/content/drive/MyDrive/seokwon/valid_3'

# categories = ['얼룩', '주름', '틈']

# for category in categories:
#     category_folder = os.path.join(data_path, category)
#     subcategories = os.listdir(category_folder)
#     for subcategory in subcategories:
#         subcategory_folder = os.path.join(category_folder, subcategory)
#         img_names = sorted(os.listdir(subcategory_folder))
#         img_count = len(img_names)
#         train_count = int(0.8 * img_count)  # train 데이터의 비율을 설정합니다.

#         # train 폴더로 이미지 파일 복사합니다.
#         for file in img_names[:train_count]:
#             if True:  # 모든 파일을 복사하려면 조건을 True로 설정합니다.
#                 source_path = os.path.join(subcategory_folder, file)
#                 target_path = os.path.join(train_path, category, file)
#                 os.makedirs(os.path.dirname(target_path), exist_ok=True)
#                 shutil.copy(source_path, target_path)

#         # valid 폴더로 이미지 파일 복사합니다.
#         for file in img_names[train_count:]:
#             if True:  # 모든 파일을 복사하려면 조건을 True로 설정합니다.
#                 source_path = os.path.join(subcategory_folder, file)
#                 target_path = os.path.join(valid_path, category, file)
#                 os.makedirs(os.path.dirname(target_path), exist_ok=True)
#                 shutil.copy(source_path, target_path)

In [22]:
# data_path = '/content/drive/MyDrive/seokwon/train_tm'
# train_path = '/content/drive/MyDrive/seokwon/train_tm_train1'
# valid_path = '/content/drive/MyDrive/seokwon/valid_tm_valid1'

# # train/valid 폴더가 없으면 생성
# if not os.path.exists(train_path):
#     os.makedirs(train_path)
# if not os.path.exists(valid_path):
#     os.makedirs(valid_path)

# categories = os.listdir(data_path)

# for category in categories:
#     category_folder = os.path.join(data_path, category)
#     subcategories = os.listdir(category_folder)
    
#     train_folder = os.path.join(train_path, category)
#     valid_folder = os.path.join(valid_path, category)

#     if not os.path.exists(train_folder):
#         os.makedirs(train_folder)
#     if not os.path.exists(valid_folder):
#         os.makedirs(valid_folder)

#     for subcategory in subcategories:
#         subcategory_folder = os.path.join(category_folder, subcategory)
#         files = os.listdir(subcategory_folder)
#         random.shuffle(files)
    
#         train_size = int(len(files) * 0.8)
#         train_files = files[:train_size]
#         valid_files = files[train_size:]
        
#         for file in train_files:
#             src_path = os.path.join(subcategory_folder, file)
#             dst_path = os.path.join(train_folder, file)
#             shutil.copy(src_path, dst_path)
            
#         for file in valid_files:
#             src_path = os.path.join(subcategory_folder, file)
#             dst_path = os.path.join(valid_folder, file)
#             shutil.copy(src_path, dst_path)


FileNotFoundError: ignored

In [29]:
! mkdir '/content/drive/MyDrive/seokwon/train_tm_train/얼룩'
! mkdir '/content/drive/MyDrive/seokwon/valid_tm_valid/얼룩'

In [30]:
import os
import random
import shutil


# 데이터 경로 및 train/valid 폴더 경로 설정
data_path = '/content/drive/MyDrive/seokwon/train_tm/얼룩'
train_path = '/content/drive/MyDrive/seokwon/train_tm_train/얼룩'
valid_path = '/content/drive/MyDrive/seokwon/valid_tm_train/얼룩'

# train/valid 폴더가 없으면 생성
if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(valid_path):
    os.makedirs(valid_path)

# 각 카테고리 폴더 안에 있는 파일들을 불러옴
categories = os.listdir(data_path)

# 각 카테고리 폴더에서 파일을 랜덤하게 선택하여 train/valid 폴더에 복사
for category in categories:
    folder = os.path.join(data_path, category)
    train_folder = os.path.join(train_path, category)
    valid_folder = os.path.join(valid_path, category)
    
    # train/valid 폴더가 없으면 생성
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(valid_folder):
        os.makedirs(valid_folder)
        
    files = os.listdir(folder)
    random.shuffle(files)
    
    train_files = files[:int(len(files)*0.8)]
    valid_files = files[int(len(files)*0.8):]
    
    for file in train_files:
        shutil.copy(os.path.join(folder, file), os.path.join(train_folder, file))
        
    for file in valid_files:
        shutil.copy(os.path.join(folder, file), os.path.join(valid_folder, file))
    
    

In [35]:
# 데이터 경로 및 train/valid 폴더 경로 설정
data_path = '/content/drive/MyDrive/seokwon/train_tm/틈'
train_path = '/content/drive/MyDrive/seokwon/train_tm_train/틈'
valid_path = '/content/drive/MyDrive/seokwon/valid_tm_train/틈'

# train/valid 폴더가 없으면 생성
if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(valid_path):
    os.makedirs(valid_path)

# 각 카테고리 폴더 안에 있는 파일들을 불러옴
categories = os.listdir(data_path)

# 각 카테고리 폴더에서 파일을 랜덤하게 선택하여 train/valid 폴더에 복사
for category in categories:
    folder = os.path.join(data_path, category)
    train_folder = os.path.join(train_path, category)
    valid_folder = os.path.join(valid_path, category)
    
    # train/valid 폴더가 없으면 생성
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(valid_folder):
        os.makedirs(valid_folder)
        
    files = os.listdir(folder)
    random.shuffle(files)
    
    train_files = files[:int(len(files)*0.8)]
    valid_files = files[int(len(files)*0.8):]
    
    for file in train_files:
        shutil.copy(os.path.join(folder, file), os.path.join(train_folder, file))
        
    for file in valid_files:
        shutil.copy(os.path.join(folder, file), os.path.join(valid_folder, file))
    

In [36]:
# 데이터 경로 및 train/valid 폴더 경로 설정
data_path = '/content/drive/MyDrive/seokwon/train_tm/주름'
train_path = '/content/drive/MyDrive/seokwon/train_tm_train/주름'
valid_path = '/content/drive/MyDrive/seokwon/valid_tm_train/주름'

# train/valid 폴더가 없으면 생성
if not os.path.exists(train_path):
    os.makedirs(train_path)
if not os.path.exists(valid_path):
    os.makedirs(valid_path)

# 각 카테고리 폴더 안에 있는 파일들을 불러옴
categories = os.listdir(data_path)

# 각 카테고리 폴더에서 파일을 랜덤하게 선택하여 train/valid 폴더에 복사
for category in categories:
    folder = os.path.join(data_path, category)
    train_folder = os.path.join(train_path, category)
    valid_folder = os.path.join(valid_path, category)
    
    # train/valid 폴더가 없으면 생성
    if not os.path.exists(train_folder):
        os.makedirs(train_folder)
    if not os.path.exists(valid_folder):
        os.makedirs(valid_folder)
        
    files = os.listdir(folder)
    random.shuffle(files)
    
    train_files = files[:int(len(files)*0.8)]
    valid_files = files[int(len(files)*0.8):]
    
    for file in train_files:
        shutil.copy(os.path.join(folder, file), os.path.join(train_folder, file))
        
    for file in valid_files:
        shutil.copy(os.path.join(folder, file), os.path.join(valid_folder, file))
    

In [37]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir='/content/drive/MyDrive/seokwon/train_tm_train'
valid_dir='/content/drive/MyDrive/seokwon/valid_tm_train'

img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',) 

valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True) 

Found 7600 images belonging to 3 classes.
Found 1900 images belonging to 3 classes.


In [43]:
print(train_generator.class_indices)

{'얼룩': 0, '주름': 1, '틈': 2}


In [40]:
len(os.listdir('/content/drive/MyDrive/seokwon/train_tm'))

3

In [41]:
import tensorflow as tf
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from keras.callbacks import EarlyStopping


tf.keras.backend.clear_session()
# Load the EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(3, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=x)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta=0, verbose=1, restore_best_weights=True, patience=5)
# Train the model
history = model.fit(train_generator,batch_size=32, epochs=100,callbacks=[es], validation_data=valid_generator)

Epoch 1/100
238/238 [==============================] - 172s 547ms/step - loss: 0.4648 - accuracy: 0.8218 - val_loss: 0.2670 - val_accuracy: 0.9032
Epoch 2/100
238/238 [==============================] - 128s 535ms/step - loss: 0.2752 - accuracy: 0.8967 - val_loss: 0.2488 - val_accuracy: 0.9195
Epoch 3/100
238/238 [==============================] - 124s 521ms/step - loss: 0.2158 - accuracy: 0.9247 - val_loss: 0.2580 - val_accuracy: 0.9089
Epoch 4/100
238/238 [==============================] - 124s 519ms/step - loss: 0.1738 - accuracy: 0.9375 - val_loss: 0.3677 - val_accuracy: 0.8947
Epoch 5/100
238/238 [==============================] - 123s 515ms/step - loss: 0.1521 - accuracy: 0.9458 - val_loss: 0.2436 - val_accuracy: 0.9274
Epoch 6/100
238/238 [==============================] - 124s 519ms/step - loss: 0.1520 - accuracy: 0.9480 - val_loss: 0.1790 - val_accuracy: 0.9463
Epoch 7/100
238/238 [==============================] - 124s 518ms/step - loss: 0.1211 - accuracy: 0.9539 - val_loss: 0

In [42]:
model.save_weights('/content/drive/MyDrive/seokwon/tm.h5')

In [ ]:
from tensorflow.keras.models import load_model

In [44]:
test_dir = '/content/drive/MyDrive/new/test'
test_images = os.listdir(test_dir)

# 2. Loop through each image, preprocess it, and make a prediction

train_dir = '/content/drive/MyDrive/seokwon/train_tm_train'
class_names = sorted(os.listdir(train_dir))
class_indices = {name: i for i, name in enumerate(class_names)}
predictions = []
for image_file in test_images:
  image_path = os.path.join(test_dir, image_file)
  img = cv2.imread(image_path)
  img = cv2.resize(img, (224, 224))
  img = np.expand_dims(img, axis=0)
  pred = model.predict(img)
  predicted_class_idx = np.argmax(pred, axis=-1)
  predicted_class = class_indices[class_names[predicted_class_idx[0]]]
  predictions.append(predicted_class)

# 3. Save or print the results
print(predictions)

1/1 [==============================] - 0s 23ms/step
[1, 0, 1, 2, 0, 1, 0, 1, 2, 1, 2, 1, 0, 0, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1, 2, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 2, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 2, 1, 1, 0, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,

In [45]:
import pandas as pd

In [46]:
result=pd.read_csv('/content/drive/MyDrive/seokwon/sample_submission.csv',encoding='utf-8')

In [47]:
result.drop('label',axis=1,inplace=True)

In [48]:
result

,id
0,TEST_000
1,TEST_001
2,TEST_002
3,TEST_003
4,TEST_004
...,...
787,TEST_787
788,TEST_788
789,TEST_789
790,TEST_790


In [56]:
result['label']=predictions

In [57]:
map={0:'얼룩', 1: '주름', 2: '틈'}

In [ ]:
# map= {
#     0: '가구수정',
#     1: '걸레받이수정',
#     2: '곰팡이',
#     3: '꼬임',
#     4: '녹오염',
#     5: '들뜸',
#     6: '면불량',
#     7: '몰딩수정',
#     8: '반점',
#     9: '석고수정',
#     10: '오염',
#     11: '오타공',
#     12: '울음',
#     13: '이음부불량',
#     14: '창틀,문틀수정',
#     15: '터짐',
#     16: '틈새과다',
#     17: '피스',
#     18: '훼손'
# }

In [58]:
result['label']=result['label'].map(map)

In [59]:
result

,id,label
0,TEST_000,주름
1,TEST_001,얼룩
2,TEST_002,주름
3,TEST_003,틈
4,TEST_004,얼룩
...,...,...
787,TEST_787,틈
788,TEST_788,주름
789,TEST_789,얼룩
790,TEST_790,얼룩


In [60]:
result.to_csv('/content/drive/MyDrive/seokwon/tm.csv', encoding='utf-8' ,index=False)

In [61]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir='/content/drive/MyDrive/seokwon/train_tm_train/얼룩'
valid_dir='/content/drive/MyDrive/seokwon/valid_tm_train/얼룩'

img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',) 

valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True) 

Found 1600 images belonging to 4 classes.
Found 400 images belonging to 4 classes.


In [62]:
print(train_generator.class_indices)

{'곰팡이': 0, '녹오염': 1, '반점': 2, '오염': 3}


In [64]:
tf.keras.backend.clear_session()
# Load the EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(4, activation='softmax')(x)
model_ul = Model(inputs=base_model.input, outputs=x)

# Compile the model
model_ul.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta=0, verbose=1, restore_best_weights=True, patience=5)
# Train the model
history = model_ul.fit(train_generator,batch_size=32, epochs=100,callbacks=[es], validation_data=valid_generator)

Epoch 1/100
50/50 [==============================] - 69s 619ms/step - loss: 0.3053 - accuracy: 0.8894 - val_loss: 0.3097 - val_accuracy: 0.9275
Epoch 2/100
50/50 [==============================] - 29s 578ms/step - loss: 0.1046 - accuracy: 0.9675 - val_loss: 0.3332 - val_accuracy: 0.9475
Epoch 3/100
50/50 [==============================] - 30s 586ms/step - loss: 0.0750 - accuracy: 0.9737 - val_loss: 0.2938 - val_accuracy: 0.9450
Epoch 4/100
50/50 [==============================] - 29s 585ms/step - loss: 0.0537 - accuracy: 0.9862 - val_loss: 0.1043 - val_accuracy: 0.9700
Epoch 5/100
50/50 [==============================] - 30s 591ms/step - loss: 0.0545 - accuracy: 0.9812 - val_loss: 0.1200 - val_accuracy: 0.9800
Epoch 6/100
50/50 [==============================] - 29s 574ms/step - loss: 0.0683 - accuracy: 0.9812 - val_loss: 0.1042 - val_accuracy: 0.9800
Epoch 7/100
50/50 [==============================] - 29s 576ms/step - loss: 0.0592 - accuracy: 0.9831 - val_loss: 0.3215 - val_accuracy:

In [134]:
model_ul.save_weights('/content/drive/MyDrive/seokwon/tm_ul.h5')

In [112]:
result=pd.read_csv('/content/drive/MyDrive/seokwon/tm.csv')

In [94]:
test_dir = '/content/drive/MyDrive/new/test'
test_images = result[result['label'] == '얼룩']['id'].str.replace('TEST_', '').tolist()

class_names = sorted(os.listdir(train_dir))
class_indices = {name: i for i, name in enumerate(class_names)}
predictions = []

for image_id in test_images:
    image_file = f"{image_id}.png"  # 테스트 이미지 파일 이름 설정 (png 형식)
    image_path = os.path.join(test_dir, image_file)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    pred = model_ul.predict(img)
    predicted_class_idx = np.argmax(pred, axis=-1)
    predicted_class = class_indices[class_names[predicted_class_idx[0]]]
    predictions.append(predicted_class)

# 결과 출력
print(predictions)

1/1 [==============================] - 0s 25ms/step
[3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 0, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]


In [107]:
map={0:'곰팡이', 1: '녹오염', 2: '반점', 3: '오염'}

In [113]:
result

,id,label
0,TEST_000,주름
1,TEST_001,얼룩
2,TEST_002,주름
3,TEST_003,틈
4,TEST_004,얼룩
...,...,...
787,TEST_787,틈
788,TEST_788,주름
789,TEST_789,얼룩
790,TEST_790,얼룩


In [114]:
result.loc[result['label']=='얼룩', 'label'] = predictions

In [116]:
result['label'] = result['label'].apply(lambda x: map[x] if x in map else x)

In [117]:
result

,id,label
0,TEST_000,주름
1,TEST_001,오염
2,TEST_002,주름
3,TEST_003,틈
4,TEST_004,오염
...,...,...
787,TEST_787,틈
788,TEST_788,주름
789,TEST_789,오염
790,TEST_790,오염


In [118]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir='/content/drive/MyDrive/seokwon/train_tm_train/주름'
valid_dir='/content/drive/MyDrive/seokwon/valid_tm_train/주름'

img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',) 

valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True) 

Found 3600 images belonging to 9 classes.
Found 900 images belonging to 9 classes.


In [119]:
print(train_generator.class_indices)

{'꼬임': 0, '면불량': 1, '석고수정': 2, '오타공': 3, '울음': 4, '이음부불량': 5, '터짐': 6, '피스': 7, '훼손': 8}


In [121]:
tf.keras.backend.clear_session()
# Load the EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(9, activation='softmax')(x)
model_ju = Model(inputs=base_model.input, outputs=x)

# Compile the model
model_ju.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta=0, verbose=1, restore_best_weights=True, patience=5)
# Train the model
history = model_ju.fit(train_generator,batch_size=32, epochs=100,callbacks=[es], validation_data=valid_generator)

Epoch 1/100
113/113 [==============================] - 98s 545ms/step - loss: 0.8924 - accuracy: 0.7031 - val_loss: 1.1081 - val_accuracy: 0.7244
Epoch 2/100
113/113 [==============================] - 60s 528ms/step - loss: 0.4087 - accuracy: 0.8642 - val_loss: 0.4711 - val_accuracy: 0.8556
Epoch 3/100
113/113 [==============================] - 60s 529ms/step - loss: 0.2594 - accuracy: 0.9042 - val_loss: 0.4533 - val_accuracy: 0.8856
Epoch 4/100
113/113 [==============================] - 60s 527ms/step - loss: 0.2198 - accuracy: 0.9275 - val_loss: 0.2782 - val_accuracy: 0.9189
Epoch 5/100
113/113 [==============================] - 60s 529ms/step - loss: 0.1636 - accuracy: 0.9428 - val_loss: 0.3362 - val_accuracy: 0.9156
Epoch 6/100
113/113 [==============================] - 60s 529ms/step - loss: 0.1758 - accuracy: 0.9425 - val_loss: 0.3847 - val_accuracy: 0.9133
Epoch 7/100
113/113 [==============================] - 60s 533ms/step - loss: 0.1442 - accuracy: 0.9497 - val_loss: 0.4032 -

In [135]:
model_ju.save_weights('/content/drive/MyDrive/seokwon/tm_ju.h5')

In [124]:
test_dir = '/content/drive/MyDrive/new/test'
test_images = result[result['label'] == '주름']['id'].str.replace('TEST_', '').tolist()

class_names = sorted(os.listdir(train_dir))
class_indices = {name: i for i, name in enumerate(class_names)}
predictions = []

for image_id in test_images:
    image_file = f"{image_id}.png"  # 테스트 이미지 파일 이름 설정 (png 형식)
    image_path = os.path.join(test_dir, image_file)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    pred = model_ju.predict(img)
    predicted_class_idx = np.argmax(pred, axis=-1)
    predicted_class = class_indices[class_names[predicted_class_idx[0]]]
    predictions.append(predicted_class)

# 결과 출력
print(predictions)

1/1 [==============================] - 0s 24ms/step
[8, 2, 8, 8, 3, 3, 8, 1, 8, 6, 8, 8, 6, 8, 1, 6, 8, 8, 8, 8, 8, 8, 8, 8, 8, 0, 6, 0, 8, 8, 8, 8, 8, 5, 1, 8, 8, 8, 2, 3, 8, 8, 8, 8, 8, 6, 8, 8, 8, 6, 8, 8, 1, 3, 8, 8, 6, 8, 0, 8, 8, 8, 8, 6, 8, 8, 8, 7, 3, 7, 8, 8, 6, 8, 8, 1, 0, 8, 8, 8, 8, 8, 8, 8, 8, 8, 7, 8, 8, 8, 2, 8, 8, 8, 6, 8, 2, 0, 8, 1, 8, 8, 0, 8, 8, 8, 8, 8, 8, 6, 8, 8, 8, 8, 5, 0, 8, 8, 8, 8, 1, 8, 8, 8, 8, 8, 8, 8, 0, 8, 8, 0, 8, 8, 0, 8, 8, 8, 8, 0, 8, 8, 8, 8, 8, 8, 8, 5, 8, 0, 0, 0, 6, 0, 8, 0, 8, 6, 8, 8, 8, 8, 6, 8, 8, 8, 6, 8, 2, 6, 8, 0, 0, 8, 1, 8, 1, 8, 8, 8, 8, 8, 8, 3, 6, 3, 8, 8, 8, 8, 7, 8, 8, 3, 8, 8, 6, 8, 2, 2, 1, 6, 3, 8, 6, 0, 0, 6, 8, 7, 6, 8, 8, 8, 8, 6, 8, 8, 4, 6, 8, 8, 8, 8, 8, 6, 7, 8, 0, 8, 8, 8, 7, 7, 8, 8, 5, 8, 8, 2, 2, 6, 0, 8, 8, 6, 1, 8, 6, 5, 8, 8, 8, 8, 8, 8, 3, 3, 8, 6, 7, 8, 6, 8, 8, 6, 6, 8, 0, 8, 8, 8, 8, 8, 8, 6, 3, 7, 6, 1, 8, 6, 8, 8, 8, 8, 8, 8, 5, 7, 8, 2, 1, 8, 6, 1, 6, 8, 8, 1, 8, 1, 8, 7, 0, 8, 8, 8, 8, 8, 0, 0, 8, 8, 8, 8,

In [125]:
map={0:'꼬임', 1: '면불량', 2: '석고수정', 3:'오타공', 4: '울음', 5: '이음부불량', 6: '터짐', 7:'피스', 8: '훼손'}

In [126]:
result.loc[result['label']=='주름', 'label'] = predictions

In [128]:
result['label'] = result['label'].apply(lambda x: map[x] if x in map else x)

In [130]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dir='/content/drive/MyDrive/seokwon/train_tm_train/틈'
valid_dir='/content/drive/MyDrive/seokwon/valid_tm_train/틈'

img_height, img_width = 224, 224
batch_size = 32

train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    fill_mode='nearest',) 

valid_datagen = ImageDataGenerator()

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True) 

Found 2400 images belonging to 6 classes.
Found 600 images belonging to 6 classes.


In [133]:
print(train_generator.class_indices)

{'가구수정': 0, '걸레받이수정': 1, '들뜸': 2, '몰딩수정': 3, '창틀,문틀수정': 4, '틈새과다': 5}


In [132]:
tf.keras.backend.clear_session()
# Load the EfficientNetB0 model
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add custom classification head
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(6, activation='softmax')(x)
model_tm= Model(inputs=base_model.input, outputs=x)

# Compile the model
model_tm.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

es = EarlyStopping(monitor='val_loss', min_delta=0, verbose=1, restore_best_weights=True, patience=5)
# Train the model
history = model_tm.fit(train_generator,batch_size=32, epochs=100,callbacks=[es], validation_data=valid_generator)

Epoch 1/100
75/75 [==============================] - 76s 521ms/step - loss: 0.5559 - accuracy: 0.7929 - val_loss: 0.9610 - val_accuracy: 0.8167
Epoch 2/100
75/75 [==============================] - 37s 495ms/step - loss: 0.1823 - accuracy: 0.9479 - val_loss: 0.1999 - val_accuracy: 0.9433
Epoch 3/100
75/75 [==============================] - 38s 502ms/step - loss: 0.1037 - accuracy: 0.9667 - val_loss: 0.2135 - val_accuracy: 0.9483
Epoch 4/100
75/75 [==============================] - 37s 496ms/step - loss: 0.1134 - accuracy: 0.9650 - val_loss: 0.2395 - val_accuracy: 0.9317
Epoch 5/100
75/75 [==============================] - 38s 500ms/step - loss: 0.1245 - accuracy: 0.9546 - val_loss: 0.1634 - val_accuracy: 0.9617
Epoch 6/100
75/75 [==============================] - 37s 493ms/step - loss: 0.0686 - accuracy: 0.9783 - val_loss: 0.1104 - val_accuracy: 0.9583
Epoch 7/100
75/75 [==============================] - 37s 493ms/step - loss: 0.0776 - accuracy: 0.9767 - val_loss: 0.1073 - val_accuracy:

In [136]:
model_tm.save_weights('/content/drive/MyDrive/seokwon/tm_tm.h5')

In [138]:
test_dir = '/content/drive/MyDrive/new/test'
test_images = result[result['label'] == '틈']['id'].str.replace('TEST_', '').tolist()

class_names = sorted(os.listdir(train_dir))
class_indices = {name: i for i, name in enumerate(class_names)}
predictions = []

for image_id in test_images:
    image_file = f"{image_id}.png"  # 테스트 이미지 파일 이름 설정 (png 형식)
    image_path = os.path.join(test_dir, image_file)
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    pred = model_tm.predict(img)
    predicted_class_idx = np.argmax(pred, axis=-1)
    predicted_class = class_indices[class_names[predicted_class_idx[0]]]
    predictions.append(predicted_class)

# 결과 출력
print(predictions)

1/1 [==============================] - 0s 24ms/step
[3, 3, 3, 4, 4, 3, 2, 1, 3, 3, 4, 3, 4, 1, 0, 3, 3, 0, 1, 1, 4, 3, 3, 3, 1, 3, 2, 3, 4, 1, 3, 1, 2, 3, 4, 3, 0, 2, 3, 4, 4, 3, 4, 3, 2, 4, 3, 3, 3, 4, 3, 1, 5]


In [139]:
map={0:'가구수정', 1: '걸레받이수정', 2: '들뜸', 3: '몰딩수정', 4: '창틀,문틀수정', 5: '틈새과다'}

In [140]:
result.loc[result['label']=='틈', 'label'] = predictions

In [141]:
result

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,석고수정
3,TEST_003,3
4,TEST_004,오염
...,...,...
787,TEST_787,1
788,TEST_788,터짐
789,TEST_789,오염
790,TEST_790,오염


In [142]:
result['label'] = result['label'].apply(lambda x: map[x] if x in map else x)

In [148]:
result.to_csv('/content/drive/MyDrive/seokwon/result_total.csv', encoding='utf-8' ,index=False)

In [149]:
result

,id,label
0,TEST_000,훼손
1,TEST_001,오염
2,TEST_002,석고수정
3,TEST_003,몰딩수정
4,TEST_004,오염
...,...,...
787,TEST_787,걸레받이수정
788,TEST_788,터짐
789,TEST_789,오염
790,TEST_790,오염
